# AzureOpenAIEmbeddings

这将帮助你开始使用 LangChain 的 AzureOpenAI embedding 模型。有关 `AzureOpenAIEmbeddings` 功能和配置选项的详细文档，请参阅[API 参考](https://python.langchain.com/api_reference/openai/embeddings/langchain_openai.embeddings.AzureOpenAIEmbeddings.html)。

## 概述
### 集成详情

import { ItemTable } from "@theme/FeatureTables";

<ItemTable category="text_embedding" item="AzureOpenAI" />

## 设置

要访问 AzureOpenAI embedding 模型，你需要创建一个 Azure 账户，获取一个 API 密钥，并安装 `langchain-openai` 集成包。

### 凭证

你需要部署一个 Azure OpenAI 实例。你可以在 Azure 门户上按照此[指南](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal)部署一个版本。

一旦你的实例运行起来，请确保你拥有实例的名称和密钥。你可以在 Azure 门户中找到密钥，在实例的“Keys and Endpoint”部分。

```bash
AZURE_OPENAI_ENDPOINT=<YOUR API ENDPOINT>
AZURE_OPENAI_API_KEY=<YOUR_KEY>
AZURE_OPENAI_API_VERSION="2024-02-01"
```

In [ ]:
import getpass
import os

if not os.getenv("AZURE_OPENAI_API_KEY"):
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass(
        "Enter your AzureOpenAI API key: "
    )

要启用模型调用的自动化跟踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [9]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### 安装

LangChain AzureOpenAI 集成位于 `langchain-openai` 包中：

In [ ]:
%pip install -qU langchain-openai

## 实例化

现在我们可以实例化我们的模型对象并生成聊天补全：

In [11]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    # azure_endpoint="https://<your-endpoint>.openai.azure.com/", If not provided, will read env variable AZURE_OPENAI_ENDPOINT
    # api_key=... # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
    # openai_api_version=..., # If not provided, will read env variable AZURE_OPENAI_API_VERSION
)

## 索引与检索

词嵌入模型常用于检索增强生成（RAG）流程中，既作为索引数据的组成部分，也用于后续的检索。更详细的说明，请参阅我们的 [RAG 教程](/docs/tutorials/)。

下面，展示如何使用我们上面初始化的 `embeddings` 对象来索引和检索数据。在此示例中，我们将在 `InMemoryVectorStore` 中索引和检索一个示例文档。

In [5]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content

'LangChain is the framework for building context-aware reasoning applications'

## 直接使用

在内部，vectorstore 和 retriever 实现会分别调用 `embeddings.embed_documents(...)` 和 `embeddings.embed_query(...)` 来为 `from_texts` 和检索 `invoke` 操作中使用的文本创建 embeddings。

您可以直接调用这些方法来获取 embeddings 以满足您自己的用例。

### 嵌入单个文本

您可以使用 `embed_query` 来嵌入单个文本或文档：

In [6]:
single_vector = embeddings.embed_query(text)
print(str(single_vector)[:100])  # Show the first 100 characters of the vector

[-0.0011676070280373096, 0.007125577889382839, -0.014674457721412182, -0.034061674028635025, 0.01128


### 嵌入多个文本

你可以使用 `embed_documents` 嵌入多个文本：

In [7]:
text2 = (
    "LangGraph is a library for building stateful, multi-actor applications with LLMs"
)
two_vectors = embeddings.embed_documents([text, text2])
for vector in two_vectors:
    print(str(vector)[:100])  # Show the first 100 characters of the vector

[-0.0011966148158535361, 0.007160289213061333, -0.014659193344414234, -0.03403077274560928, 0.011280
[-0.005595256108790636, 0.016757294535636902, -0.011055258102715015, -0.031094247475266457, -0.00363


## API 参考

有关 `AzureOpenAIEmbeddings` 的详细文档、功能和配置选项，请参阅[API 参考](https://python.langchain.com/api_reference/openai/embeddings/langchain_openai.embeddings.azure.AzureOpenAIEmbeddings.html)。